In [1]:
!module list


Currently Loaded Modules:
  1) anaconda3/5.1.0-gcc/8.3.1     4) cudnn/8.0.0.180-11.0-linux-x64-gcc/7.5.0
  2) anaconda3/2019.10-gcc/8.3.1   5) openjdk/1.8.0_222-b10-gcc/8.3.1
  3) cuda/11.0.3-gcc/7.5.0         6) hadoop/3.2.1-gcc/8.3.1

 



In [2]:
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Flatten, TimeDistributed, Dropout, LSTMCell, RNN, Bidirectional, Concatenate, Layer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.utils import tf_utils
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle


import unicodedata
import re
import os
import time
import shutil
import requests
import tarfile
import glob

import pandas as pd
import numpy as np
import string, os
tf.__version__

'2.4.0'

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_memory_growth(physical_devices[1], True)

In [4]:
file_name = "Project_CodeNet_LangClass.tar.gz"
data_url = f"https://dax-cdn.cdn.appdomain.cloud/dax-project-codenet/1.0.0/{file_name}"

# Download tar archive to local disk
with open(file_name, "wb") as f:
    f.write(requests.get(data_url).content)
    
# Extract contents of archive to local disk
if os.path.exists("data"):
    shutil.rmtree("data")    
with tarfile.open(file_name) as tfile:
    tfile.extractall()
    
!ls data data/train

data:
test  train

data/train:
C  C#  C++  D  Haskell	Java  JavaScript  PHP  Python  Rust


In [5]:
import os
import argparse
import re
from tokenize import tokenize, untokenize, COMMENT, STRING, NEWLINE, ENCODING, ENDMARKER, NL, INDENT, NUMBER
from io import BytesIO
import json

lits = json.load(open("literals.json"))

def process_string(token, special_chars={" ": "U+0020", ",": "U+002C"}):
    str_quote_options = ["'''", '"""', "'", '"']
    start_quote = ""
    end_quote = ""
    qualifier_regex = r"^[a-z]+"
    qualifier_match = re.search(qualifier_regex, token)
    # string qualifiers like 'r' for regex, 'f' for formatted string, 'b' for bytes, 'u' for unicode, etc (or combination of them)
    qualifier = "" if not qualifier_match else qualifier_match[0]
    # token string without qualifiers
    token_string = re.sub(qualifier_regex, "", token)
    # string literal without quotes
    str_lit = token_string
    for q in str_quote_options:
        if token_string.startswith(q):
            start_quote = q
            str_lit = str_lit[len(q) :]
            if token_string.endswith(q):
                end_quote = q
                str_lit = str_lit[: -len(q)]
            break
    # if start_quote in str_quote_options[:2]:
    #     return ""
    for sc in special_chars:
        str_lit = str_lit.replace(sc, special_chars[sc])
    return (
        f"{qualifier}{start_quote}<STR_LIT:{str_lit}>{end_quote}"
        if str_lit in lits['str']
        else f"{qualifier}{start_quote}<STR_LIT>{end_quote}"
    )

def py_tokenize(file_type):
    file_paths = glob.glob(os.path.join(os.getcwd(),"data/"+file_type+"/Python","*.*"))
    wf = open(os.path.join(os.getcwd(), f"{file_type}.txt"), 'w')
    local_corpus = []
    for path in file_paths:
        try:
            code = open(path).read()
            token_gen = tokenize(BytesIO(bytes(code, "utf8")).readline)
            out_tokens = []
            prev_eol = False
            for toknum, tokval, _, _, _ in token_gen:
                tokval = " ".join(tokval.split())
                if toknum == STRING:
                    add_token = process_string(tokval)
                    out_tokens.append(add_token)
                    prev_eol = False
                elif toknum == NUMBER:
                    if tokval in lits['num']:
                        out_tokens.append(f"<NUM_LIT:{tokval}>")
                    else:
                        out_tokens.append(f"<NUM_LIT>")
                    prev_eol = False
                elif toknum in [NEWLINE, NL]:
                    if not prev_eol:
                        out_tokens.append("<EOL>")
                        prev_eol = True
                elif toknum in [COMMENT, INDENT, ENCODING, ENDMARKER] or len(tokval) == 0:
                    continue
                else:
                    out_tokens.append(tokval)
                    prev_eol = False
            if out_tokens[0] == "<EOL>":
                out_tokens = out_tokens[1:]
            if out_tokens[-1] == "<EOL>":
                out_tokens = out_tokens[:-1]
        except Exception:
            out_tokens = []
#         local_corpus.extend((" ".join(out_tokens)).split('<EOL>'))
#         out_tokens = ["<s>"] + out_tokens + ["</s>"]
        out = " ".join(out_tokens)
        local_corpus.append(out)
        wf.write(out+"\n")
    print(f"{file_type}: are done")
    wf.close()
    return local_corpus

In [6]:
corpus = py_tokenize("train")
corpus_new = []
for code in corpus:
    corpus_new.extend(code.split('<EOL>'))
print(corpus_new[0:20])
print(len(corpus_new))

train: are done
['arr = [ <NUM_LIT:0> ] * <NUM_LIT:100> ', ' while True : ', ' try : ', ' x , y , s = map ( int , input ( ) . split ( "<STR_LIT:U+002C>" ) ) ', ' if s == <NUM_LIT:3> : ', ' if x <= <NUM_LIT:7> : ', ' arr [ <NUM_LIT:10> * y + x + <NUM_LIT:2> ] += <NUM_LIT:1> ', ' if x >= <NUM_LIT:2> : ', ' arr [ <NUM_LIT:10> * y + x - <NUM_LIT:2> ] += <NUM_LIT:1> ', ' if y <= <NUM_LIT:7> : ', ' arr [ <NUM_LIT:10> * y + x + <NUM_LIT:20> ] += <NUM_LIT:1> ', ' if y >= <NUM_LIT:2> : ', ' arr [ <NUM_LIT:10> * y + x - <NUM_LIT:20> ] += <NUM_LIT:1> ', ' if s >= <NUM_LIT:2> : ', ' if x != <NUM_LIT:9> and y != <NUM_LIT:9> : ', ' arr [ <NUM_LIT:10> * y + x + <NUM_LIT:11> ] += <NUM_LIT:1> ', ' if x != <NUM_LIT:9> and y != <NUM_LIT:0> : ', ' arr [ <NUM_LIT:10> * y + x - <NUM_LIT:9> ] += <NUM_LIT:1> ', ' if x != <NUM_LIT:0> and y != <NUM_LIT:0> : ', ' arr [ <NUM_LIT:10> * y + x - <NUM_LIT:11> ] += <NUM_LIT:1> ']
3815


In [7]:
full_corpus_tokens = ''.join(corpus).split()
print(full_corpus_tokens[0:100])

['arr', '=', '[', '<NUM_LIT:0>', ']', '*', '<NUM_LIT:100>', '<EOL>', 'while', 'True', ':', '<EOL>', 'try', ':', '<EOL>', 'x', ',', 'y', ',', 's', '=', 'map', '(', 'int', ',', 'input', '(', ')', '.', 'split', '(', '"<STR_LIT:U+002C>"', ')', ')', '<EOL>', 'if', 's', '==', '<NUM_LIT:3>', ':', '<EOL>', 'if', 'x', '<=', '<NUM_LIT:7>', ':', '<EOL>', 'arr', '[', '<NUM_LIT:10>', '*', 'y', '+', 'x', '+', '<NUM_LIT:2>', ']', '+=', '<NUM_LIT:1>', '<EOL>', 'if', 'x', '>=', '<NUM_LIT:2>', ':', '<EOL>', 'arr', '[', '<NUM_LIT:10>', '*', 'y', '+', 'x', '-', '<NUM_LIT:2>', ']', '+=', '<NUM_LIT:1>', '<EOL>', 'if', 'y', '<=', '<NUM_LIT:7>', ':', '<EOL>', 'arr', '[', '<NUM_LIT:10>', '*', 'y', '+', 'x', '+', '<NUM_LIT:20>', ']', '+=', '<NUM_LIT:1>', '<EOL>', 'if', 'y']


In [8]:
train_len = 3+1
text_sequences = []
for i in range(train_len,len(full_corpus_tokens)):
    seq = full_corpus_tokens[i-train_len:i]
    text_sequences.append(seq)
sequences = {}
count = 1
for i in range(len(full_corpus_tokens)):
    if full_corpus_tokens[i] not in sequences:
        sequences[full_corpus_tokens[i]] = count
        count += 1
print(text_sequences[:10])

[['arr', '=', '[', '<NUM_LIT:0>'], ['=', '[', '<NUM_LIT:0>', ']'], ['[', '<NUM_LIT:0>', ']', '*'], ['<NUM_LIT:0>', ']', '*', '<NUM_LIT:100>'], [']', '*', '<NUM_LIT:100>', '<EOL>'], ['*', '<NUM_LIT:100>', '<EOL>', 'while'], ['<NUM_LIT:100>', '<EOL>', 'while', 'True'], ['<EOL>', 'while', 'True', ':'], ['while', 'True', ':', '<EOL>'], ['True', ':', '<EOL>', 'try']]


In [9]:
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

In [10]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
train_inputs.shape
#print(train_targets[0])

(40546, 3)

In [11]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,epochs=50,verbose=1)
model.save("mymodel_lstm.h5")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 3)              2322      
_________________________________________________________________
lstm (LSTM)                  (None, 3, 50)             10800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dense_1 (Dense)              (None, 774)               39474     
Total params: 75,346
Trainable params: 75,346
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
1268/1268 [==============================] - 9s 5ms/step - loss: 4.6838 - accuracy: 0.0875
Epoch 2/50


In [12]:
model.save('lstm_port.h5')
model = load_model('lstm_port.h5')

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

input_text = input().strip().lower()
while(input_text != 'stop'):
    encoded_text = tokenizer.texts_to_sequences([input_text])[0]
    pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
#     print(encoded_text, pad_encoded)
    predictions = []
    for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
        pred_word = tokenizer.index_word[i]
        predictions.append(pred_word)
    print("Next word suggestion:",predictions)
    input_text = input().strip().lower()

 if x


Next word suggestion: ['+', 'in', '==']


 for i


Next word suggestion: ['in', ',', '(']


 for i,


...

 import sys,


Next word suggestion: ["'<str_lit>'", '[', 'input']


 return


Next word suggestion: ['[', 'int', 'sys']


 stop
